# ROB2004 Final Project - pushing an object

The goal of this project is to resolve a simple manipulation task.

## Instructions
* Answer all questions in the notebook
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a 2-3 pages report in pdf format (pdf only) detailing the methodology you followed to answer the questions as well as answers to the questions that require a typeset answer. You may add the plots in the report (does not count for the page limit) or in the Jupyter notebook.
* This is an individual project

We will use a model of the [Kuka iiwa 14 robot](https://www.kuka.com/en-us/products/robotics-systems/industrial-robots/lbr-iiwa). This robot has 7 revolute joints and its kinematics is described in the picture below:

You will need to install pybullet on your computer. Please reach out as soon as possible for help if you do not know how to do it.

<div>
<img src="./kuka_kinematics.png" width="300"/>
</div>

# Question

Implement a controller using the code skeletong below in order to get the robot to move the yellow duck into the bin. The duck position with respect to the S frame is $(-0.2,0.6,0.55)$

<div>
<img src="./manipulation_scene.png" width="300"/>
</div>

You are free to use the methods that you want with the following constraints:
* You need to use at least one controller in the end-effector space
* You need to generate smooth motions
* You need to compensate for the gravity of the robot

In your report, describe the controller and justify your choices. Analyze the behavior of the system (include plots of the end-effector trajectories, joint trajectories, etc as you see fit).

In [1]:
import numpy as np
import time

import matplotlib.pyplot as plt

from iiwa import Simulator, iiwaRobot

np.set_printoptions(precision=3, suppress = True)

pybullet build time: Apr 21 2022 10:24:34


# Helper functions

We provide a set of helper functions (forward kinematics, Jacobians, gravity terms) that can be used to implement the desired controllers. Examples are shown below:

In [2]:
# create an instance of the helper class
my_robot = iiwaRobot()

# compute the forward kinematics for an arbitrary joint configuration
q = np.random.uniform(-1,1,7)
pose = my_robot.FK(q)
print(f'the pose of the end-effector for joint configuration\n\n {q}\n\n is\n\n {pose}')

the pose of the end-effector for joint configuration

 [-0.678 -0.673 -0.556 -0.611 -0.247 -0.346  0.076]

 is

 [[ 0.169  0.923 -0.345 -0.362]
 [-0.952  0.243  0.185  0.125]
 [ 0.254  0.297  0.92   1.177]
 [ 0.     0.     0.     1.   ]]


In [3]:
# we can also get the Jacobian in various frames 
# it is sufficient to pass the name of the frame as an argument as shown below

q = np.random.uniform(-1,1,7)

spatial_jacobian = my_robot.get_jacobian(q, 'S')
body_jacobian = my_robot.get_jacobian(q, 'B')

# we can even get the Jacobian in a frame place at the same position as the end-effector 
# frame but oriented like the spatial frame
oriented_jacobian = my_robot.get_jacobian(q, 'O')

print(f'for joint configuration \n\n {q} \n \n the spatial Jacobian is \n\n {spatial_jacobian}\n\n')
print(f'the body Jacobian is \n\n {body_jacobian}\n\n')
print(f'and the Jacobian in a frame placed at the same position as the end-effector but oriented like the spatial frame is\n\n {oriented_jacobian}')

for joint configuration 

 [ 0.881 -0.764  0.859  0.314 -0.4   -0.799 -0.837] 
 
 the spatial Jacobian is 

 [[ 0.    -0.771 -0.44   0.852 -0.331 -0.943 -0.201]
 [ 0.     0.636 -0.533  0.006 -0.768  0.244 -0.96 ]
 [ 1.     0.     0.722  0.524  0.548 -0.228 -0.195]
 [ 0.    -0.229  0.192 -0.121  0.387 -0.094  0.951]
 [ 0.    -0.278 -0.158  0.662 -0.118 -0.904 -0.239]
 [ 0.     0.    -0.     0.19   0.068 -0.578  0.198]]


the body Jacobian is 

 [[ 0.808 -0.576  0.5    0.866  0.48  -0.743  0.   ]
 [ 0.556  0.678  0.734 -0.415  0.532  0.67   0.   ]
 [-0.195 -0.456  0.46  -0.279  0.698 -0.     1.   ]
 [ 0.431  0.64   0.196 -0.107  0.064  0.081  0.   ]
 [-0.558  0.159 -0.112 -0.4   -0.058  0.09  -0.   ]
 [ 0.198 -0.573 -0.034  0.264 -0.    -0.     0.   ]]


and the Jacobian in a frame placed at the same position as the end-effector but oriented like the spatial frame is

 [[ 0.    -0.771 -0.44   0.852 -0.331 -0.943 -0.201]
 [ 0.     0.636 -0.533  0.006 -0.768  0.244 -0.96 ]
 [ 1.     0.    

In [4]:
# finally it is possible to compute the effect of gravity on the joints
# this can for example be used to do gravity compensation when controlling the robot

q = np.random.uniform(-1,1,7)

g = my_robot.g(q)

print(f'for joint configuration \n\n {q} \n \n the gravity force seen on the joints is \n \n {g}')

for joint configuration 

 [-0.483  0.342 -0.243  0.035  0.476 -0.931  0.08 ] 
 
 the gravity force seen on the joints is 
 
 [ -0.    -24.731  -0.425   7.727  -0.405   0.218   0.   ]


# Code Skeleton for the simulation

Feel free to change the `run_time` variable to match your needs.

In [ ]:
# here we create a "simulator object"
simulator = Simulator()
simulator.reset_state([0.1,0.1,0.1,0.1,0.1,0.1,0.1])


my_robot = iiwaRobot()

# duration of the simulation
run_time = 10.

dt = 0.005
num_steps = int(run_time/dt)


# we store information
ndofs = 7
measured_positions = np.zeros([num_steps,ndofs])
measured_velocities = np.zeros_like(measured_positions)
desired_torques = np.zeros_like(measured_positions)
desired_positions = np.zeros_like(measured_positions)
desired_velocities = np.zeros_like(measured_positions)
time = np.zeros([num_steps])


for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q, dq = simulator.get_state()
    measured_positions[i,:] = q
    measured_velocities[i,:] = dq
    
    ## controller: TODO HERE IMPLEMENT YOUR CONTROLLER TO SOLVE THE TASK
    ## you will need to replace the naive PD controller implemented below

    q_des = np.zeros([7])
    dq_des = np.zeros([7])
    
    # we save the desired positions/velocities for later plotting 
    desired_positions[i,:] = q_des
    desired_velocities[i,:] = dq_des
    
    ##PD controller
    # the PD gains
    P = np.array([100., 100., 100., 100., 100., 50., 50.])
    D = np.array([2.,2,2,2,2,1,1.])

    error = q_des - q # the position error for all the joints (it's a 3D vector)
    d_error = dq_des-dq # the velocity error for all the joints
    
    # we compute the desired torques as a PD controller
    joint_torques = np.diag(P) @ error + np.diag(D) @ d_error
    desired_torques[i,:] = joint_torques 
    
    # we send them to the robot and do one simulation step
    simulator.send_joint_torque(joint_torques)
    simulator.step()